<a href="https://colab.research.google.com/github/satyam123457/Dsg-project-final-/blob/main/satyam_sinha_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**WE WILL BE FIRST STARTING WITH FIRST TWO AIMS OF OUR PROJECT** 
which are :-

 


1.Recommending the 10 most popular songs 

2.Recommending top 10 user songs based on user id 

Recommenders.py is also an algorithm used here,code for which is  provided in the dsg end evaluation doc file 

In [31]:
import pandas as pd
import numpy as np
import Recommenders as Recommenders

import tensorflow as tf


*Importing the first 3 algorithms -- cosine similiarity algorithm, count vectoriser and TSNE*

## Displaying the 2 datasets 

In [32]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

 
import warnings
warnings.filterwarnings('ignore')

#next line 
song_df_1 = pd.read_csv('triplets_file.csv')
song_df_1.head()



,user_id,song_id,listen_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1.0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2.0
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1.0
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1.0
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1.0


In [33]:
song_df_2 = pd.read_csv('song_data1.csv')
song_df_2.head()


,song_id,title,release,artist_name,year
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003.0
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995.0
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006.0
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003.0
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0.0


Combine both datasets based on a common column of **song_id**

In [34]:
# combine both data
song_df = pd.merge(song_df_1, song_df_2.drop_duplicates(['song_id']), on='song_id', how='left')
song_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1.0,The Cove,Thicker Than Water,Jack Johnson,0.0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2.0,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976.0
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1.0,Stronger,Graduation,Kanye West,2007.0
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1.0,NaN,NaN,NaN,NaN
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1.0,NaN,NaN,NaN,NaN


 creating new feature combining **title** and **artist_name**




In [35]:

song_df['song'] = song_df['title']+' - '+song_df['artist_name']
song_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year,song
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1.0,The Cove,Thicker Than Water,Jack Johnson,0.0,The Cove - Jack Johnson
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2.0,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976.0,Entre Dos Aguas - Paco De Lucia
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1.0,Stronger,Graduation,Kanye West,2007.0,Stronger - Kanye West
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1.0,NaN,NaN,NaN,NaN,NaN
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1.0,NaN,NaN,NaN,NaN,NaN


# taking top 10k samples for quick results

In [36]:
song_df = song_df.head(10000)


# cummulative sum of listen count of the songs

In [37]:

song_grouped = song_df.groupby(['song']).agg({'listen_count':'count'}).reset_index()
song_grouped.head()

,song,listen_count
0,#40 - DAVE MATTHEWS BAND,1
1,'97 Bonnie & Clyde - Eminem,2
2,(Anaesthesia) Pulling Teath - Metallica,1
3,(I Cant Get No) Satisfaction - Cat Power,1
4,(I Can't Get Me No) Satisfaction - Devo,1


#Taking sum of listen count and finding the percentage listen count of every user id 

Sorting the percentages

In [38]:
grouped_sum = song_grouped['listen_count'].sum()
song_grouped['percentage'] = (song_grouped['listen_count'] / grouped_sum ) * 100
song_grouped.sort_values(['listen_count', 'song'], ascending=[0,1])

,song,listen_count,percentage
2928,Undo - Björk,32,0.513067
3195,You're The One - Dwight Yoakam,32,0.513067
683,Dog Days Are Over (Radio Edit) - Florence + Th...,28,0.448934
2737,The Scientist - Coldplay,27,0.432900
2950,Use Somebody - Kings Of Leon,27,0.432900
...,...,...,...
3216,Zero - The Pain Machinery,1,0.016033
3218,Zopf: Pigtail - Penguin Café Orchestra,1,0.016033
3221,high fives - Four Tet,1,0.016033
3223,paranoid android - Christopher O'Riley,1,0.016033


# display the top 10 popular songs

In [39]:

pr = Recommenders.popularity_recommender_py()
pr.create(song_df, 'user_id', 'song')

top_10_list =  pr.recommend(song_df['user_id'][5])
top_10_list.song.values[0]


'Undo - Björk'

aim 2 of the project (based on user id ) started 

In [40]:
# Connecting to anvil
#pip install anvil-uplink

In [41]:
pip install anvil-uplink

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [42]:
import anvil.server

anvil.server.connect("server_O2KHEW6U5J4PKMJEHPRDNFCZ-3DUBPQGSBW2TBSJJ")

In [43]:
@anvil.server.callable
def recommend_top_10(user_id):
  return top_10_list.song.values




In [44]:
ir = Recommenders.item_similarity_recommender_py()
ir.create(song_df, 'user_id', 'song')
user_items = ir.get_user_items(song_df['user_id'][5])
# display user songs history
for user_item in user_items:
  print(user_item) 
 

The Cove - Jack Johnson
Entre Dos Aguas - Paco De Lucia
Stronger - Kanye West
nan
Apuesta Por El Rock 'N' Roll - Héroes del Silencio
Paper Gangsta - Lady GaGa
Stacked Actors - Foo Fighters
Heaven's gonna burn your eyes - Thievery Corporation feat. Emiliana Torrini
Let It Be Sung - Jack Johnson / Matt Costa / Zach Gill / Dan Lebowitz / Steve Adams
I'll Be Missing You (Featuring Faith Evans & 112)(Album Version) - Puff Daddy
Love Shack - The B-52's
The Old Saloon - The Lonely Island
Behind The Sea [Live In Chicago] - Panic At The Disco
Champion - Kanye West
Country Road - Jack Johnson / Paula Fuga
Oh No - Andrew Bird
Love Song For No One - John Mayer
Jewels And Gold - Angus & Julia Stone
Neon - John Mayer
The Middle - Jimmy Eat World
High and dry - Jorge Drexler
The Christmas Song  (LP Version) - King Curtis
Are You In? - Incubus
Drive - Incubus
He Doesn't Know Why - Fleet Foxes
Sun Giant - Fleet Foxes
Right Back - Sublime
Holes To Heaven - Jack Johnson


In [45]:
based_userid_list =  ir.recommend(song_df['user_id'][5])
based_userid_list.song.values[0]

No. of unique songs for the user: 28
no. of unique songs in the training set: 3229
Non zero values in cooccurence_matrix :2202


'Tiger Mountain Peasant Song - Fleet Foxes'

**# give song recommendation for that user based on user id**


In [46]:
@anvil.server.callable
def based_userid(user_id):
  return based_userid_list.song.values
  

In [47]:
ir.recommend(song_df['user_id'][5])

No. of unique songs for the user: 28
no. of unique songs in the training set: 3229
Non zero values in cooccurence_matrix :2202


,user_id,song,score,rank
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Tiger Mountain Peasant Song - Fleet Foxes,0.032738,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Sun It Rises - Fleet Foxes,0.032738,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,I Don't Need A Man - The Pussycat Dolls,0.031888,3
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Like The Sea - Alicia Keys,0.031888,4
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Magic - The Pussycat Dolls,0.031378,5
5,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Everyone's At It - Lily Allen,0.030102,6
6,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Doesn't Mean Anything - Alicia Keys,0.027268,7
7,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Whataya Want From Me - Adam Lambert,0.026560,8
8,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Lesson Learned - Alicia Keys featuring John Mayer,0.025638,9
9,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Drunken Lullabies - Flogging Molly,0.025638,10




```
# This is formatted as code
```

#3rd aim  initialise - recommend songs based on songs enter 


In [48]:
spoti = pd.read_csv("genres_v21.csv", encoding='utf-8', quotechar='"')
spoti.head(3)



,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,id,Unnamed: 13,Unnamed: 14,Unnamed: 15,duration_ms,time_signature,genre,song_name,Unnamed: 0,title
0,0.831,0.814,2,-7.364,1,0.4200,0.0598,0.013400,0.0556,0.3890,...,2Vc6NJ9PW9gD9q343XFRKx,NaN,NaN,NaN,124539,4,Dark Trap,Mercury: Retrograde,NaN,NaN
1,0.719,0.493,8,-7.230,1,0.0794,0.4010,0.000000,0.1180,0.1240,...,7pgJBLVz5VmnL7uGHmRj6p,NaN,NaN,NaN,224427,4,Dark Trap,Pathology,NaN,NaN
2,0.850,0.893,5,-4.783,1,0.0623,0.0138,0.000004,0.3720,0.0391,...,0vSWgAlfpye0WCGeNmuNhy,NaN,NaN,NaN,98821,4,Dark Trap,Symbiote,NaN,NaN


In [49]:
spoti.shape

(42305, 22)

In [50]:
song_name = spoti["song_name"]
print(song_name.shape)
print(song_name.isnull().values.any())

(42305,)
True


In [51]:
song_name = song_name.values.reshape(-1,1)
song_name.shape

(42305, 1)

USING SIMPLE IMPUTER TO CLEAR OUT NULL VALUES

In [52]:
from sklearn.impute import SimpleImputer
imr = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imr = imr.fit(song_name)
imputed_data = imr.transform(song_name)
song_name = pd.DataFrame(imputed_data)
song_name = song_name.rename(columns={0:"Song-Names"})
song_name.head(3)

,Song-Names
0,Mercury: Retrograde
1,Pathology
2,Symbiote


In [53]:
core = spoti[["genre","mode","duration_ms"]]
print(core.dtypes)
print(core.head())

genre          object
mode            int64
duration_ms     int64
dtype: object
       genre  mode  duration_ms
0  Dark Trap     1       124539
1  Dark Trap     1       224427
2  Dark Trap     1        98821
3  Dark Trap     1       123661
4  Dark Trap     1       123298


In [54]:
core.dtypes


genre          object
mode            int64
duration_ms     int64
dtype: object

In [55]:
core["genre"].value_counts()

Underground Rap    5875
Dark Trap          4578
Hiphop             3028
trance             2999
trap               2987
techhouse          2975
dnb                2966
psytrance          2961
techno             2956
hardstyle          2936
RnB                2099
Trap Metal         1956
Rap                1848
Emo                1680
Pop                 461
Name: genre, dtype: int64

In [56]:
core = core.replace({"genre":{"Underground Rap":0, "Dark Trap":1, "Hiphop":2, "trance":3, "trap":4, "techhouse":5, "dnb":6, "psytrance": 7, "techno":8, "hardstyle":9, "RnB":10, "Trap Metal":11, "Rap":12, "Emo":13, "Pop":14}})

USING NON NEGATIVE MATRIX FACTORIZATION

In [57]:
from sklearn.decomposition import NMF
nmf = NMF(n_components = 6)


In [58]:
nmf_features = nmf.fit_transform(core)
from sklearn.preprocessing import normalize
norm_features = normalize(nmf_features)
current_music = norm_features[23,:]
similarities = norm_features.dot(current_music)
similarities

array([0.17325073, 0.36192708, 0.18100273, ..., 0.20523285, 0.1340527 ,
       0.50418458])

In [59]:
df = pd.DataFrame(norm_features)
x = df.join(song_name)
df = pd.pivot_table(x, x[[0,1,2,3,4,5]],["Song-Names"])#for indexing song_name to our df
def current_music(value):
    print("Top 5 recommendations for given music are:")
    value = df.loc[value]
    similarities = df.dot(value)
    # print(similarities.nlargest())
    return similarities.nlargest()

In [60]:
@anvil.server.callable
def current_music_2(value):
  similarity_list = current_music(value)
  # similarities = df.dot(value)
  return similarity_list.index.values

Recommending the final results based on song entered

In [61]:
x = current_music("Missed Calls - Remix")
x.index.values

Top 5 recommendations for given music are:


array(['Missed Calls - Remix', "Its Bath Time, Don't Touch My Duck",
       'We Own The Night', 'Passion', 'PILLOWTALK'], dtype=object)

In [62]:
current_music("SEARCH & DESTRXY.")

Top 5 recommendations for given music are:


Song-Names
SEARCH & DESTRXY.           1.000000
Warzone Freestyle           0.998122
SQUAD UP                    0.997902
Let's Pretend We're Numb    0.996921
ICanSeeMyHouseFromHere      0.996908
dtype: float64

In [63]:
current_music("Animals")

Top 5 recommendations for given music are:


Song-Names
Hello Fascination                                              0.989915
Rooftops (A Liberation Broadcast)                              0.989866
I Need a Girl (Pt. 2) [feat. Loon, Ginuwine & Mario Winans]    0.989844
Bugatti                                                        0.989840
Trapped in My Mind                                             0.989831
dtype: float64

In [64]:
#anvil.server.wait_forever()